### Package installation

In [ ]:
pip install psycopg2

In [ ]:
pip install pandas

In [ ]:
pip install openpyxl

### Import required Packages

In [2]:
import psycopg2
import pandas as pd
import psycopg2.extras

### Extract

***Read from excel file***

In [8]:
DATA_DIR = "/eSRD-Lab/HDA/dataset/"

In [11]:
trans_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=1, engine='openpyxl')

In [12]:
trans_dim

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


### load Trans_dim data from xlsx

In [5]:
trans_dim = pd.read_excel (r'C:\Users\HP\dataset\e-commerece_data.xlsx', sheet_name='Trans_dim')
trans_dim.head(10)

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


In [7]:
#checking for null values
trans_dim.isna().sum()

payment_key    0
trans_type     0
bank_name      0
dtype: int64

### load Store_dim data from xlsx

In [8]:
store_df = pd.read_excel (r'C:\Users\HP\dataset\e-commerece_data.xlsx', sheet_name='Store_dim')
store_df.head(5)

,store_key,division,district,upazila
0,S0001,SYLHET,HABIGANJ,AJMIRIGANJ
1,S0002,SYLHET,HABIGANJ,BAHUBAL
2,S0003,SYLHET,HABIGANJ,BANIACHONG
3,S0004,SYLHET,HABIGANJ,CHUNARUGHAT
4,S0005,SYLHET,HABIGANJ,HABIGANJ SADAR


In [9]:
store_df.dtypes

store_key    object
division     object
district     object
upazila      object
dtype: object

In [10]:
#checking for null values
store_df.isna().sum()

store_key    0
division     0
district     0
upazila      0
dtype: int64

### load Customer data

In [11]:
Coustomer_dim = pd.read_excel (r'C:\Users\HP\dataset\e-commerece_data.xlsx', sheet_name='Coustomer_dim')
Coustomer_dim.head(5)

,coustomer_key,name,contact_no,nid
0,C000001,sumit,8801920345851,7505075708899
1,C000002,tammanne,8801817069329,1977731324842
2,C000003,kailash kumar,8801663795774,3769494056318
3,C000004,bhagwati prasad,8801533627961,9378834712725
4,C000005,ajay,8801943715786,3540815556323


In [12]:
Coustomer_dim.dtypes

coustomer_key    object
name             object
contact_no        int64
nid               int64
dtype: object

In [16]:
#checking for null values
store_df.isnull().sum()

store_key    0
division     0
district     0
upazila      0
dtype: int64

### Load Item data 

In [14]:
item_dim = pd.read_excel (r'C:\Users\HP\dataset\e-commerece_data.xlsx', sheet_name='Item_dim')
item_dim.head(10)

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
5,I00006,Coke Classic 12 oz cans,a. Beverage - Soda,16.25,Lithuania,BIGSO AB,cans
6,I00007,Coke Classic 12 oz cans,a. Beverage - Soda,6.75,India,Indo Count Industries Ltd,cans
7,I00008,Coke Zero Sugar 12 oz cans,a. Beverage - Soda,16.25,India,Indo Count Industries Ltd,cans
8,I00009,Diet Coke - 12 oz cans,a. Beverage - Soda,16.25,Netherlands,Bolsius Boxmeer,cans
9,I00010,Diet Coke - 12 oz cans,a. Beverage - Soda,6.75,Lithuania,BIGSO AB,cans


In [24]:
#checking for null values
item_dim.isna().sum()

item_key       0
item_name      0
desc           0
unit_price     0
man_country    0
supplier       0
unit           1
dtype: int64

## Handle missing values

In [25]:
# Fill missing values with the most frequent value
item_dim = item_dim.fillna(item_dim.mode().iloc[0])


In [26]:
item_dim.isnull().sum()

item_key       0
item_name      0
desc           0
unit_price     0
man_country    0
supplier       0
unit           0
dtype: int64

In [ ]:
rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)

### Transform

In [37]:
# trans from item_dim
item_dim["desc"][:1].head()


0    a. Beverage - Soda
1    a. Beverage - Soda
2    a. Beverage - Soda
3    a. Beverage - Soda
4    a. Beverage - Soda
Name: desc, dtype: object

In [35]:
item_dim["desc"].str.startwith("a.","")

C:\Users\HP\AppData\Local\Temp\ipykernel_4924\615992111.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  item_dim["desc"].str.replace("a.","")


0         Bevere - Soda
1         Bevere - Soda
2         Bevere - Soda
3         Bevere - Soda
4         Bevere - Soda
             ...       
259    Kitchen Supplies
260    Kitchen Supplies
261    Kitchen Supplies
262            Medicine
263            Medicine
Name: desc, Length: 264, dtype: object

In [39]:
item_dim["desc"].rsplit("a.",1)

AttributeError: 'Series' object has no attribute 'rsplit'

### Load

***Connect to the database***

In [32]:

class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "afra1234",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [33]:
con = PostgresConnection().getConnection()

successfully connected to database


In [ ]:
insert_stmt = "INSERT INTO ecom_schema.trans_dim ( payment_key, trans_type, bank_name)" \
            " VALUES (%s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, trans_dim.values)
con.commit()
cur.close()

In [17]:
insert_stmt = "INSERT INTO ecom_schema.store_dim ( store_key, division, district, upazila)" \
            " VALUES (%s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, store_df.values)
con.commit()
cur.close()

In [23]:
insert_stmt = "INSERT INTO ecom_schema.coustomer_dim ( coustomer_key, name, contact_no, nid)" \
            " VALUES (%s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, Coustomer_dim.values)
con.commit()
cur.close()

In [31]:
insert_stmt = "INSERT INTO ecom_schema.item_dim ( item_key, item_name, desc, unit_price, man_country, supplier, unit)" \
            " VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, item_dim.values)
con.commit()
cur.close()

SyntaxError: syntax error at or near "desc"
LINE 1: ... INTO ecom_schema.item_dim ( item_key, item_name, desc, unit...
                                                             ^


In [25]:
cur = con.cursor()
select_stmt = "SELECT t.payment_key , t.trans_type, t.bank_name " \
              "FROM ecomdb_star_schema.trans_dim t"
cur.execute(select_stmt)
records = cur.fetchall()
records

[('P001', 'cash', 'None'),
 ('P002', 'card', 'AB Bank Limited'),
 ('P003', 'card', 'Bangladesh Commerce Bank Limited'),
 ('P004', 'card', 'Bank Asia Limited'),
 ('P005', 'card', 'BRAC Bank Limited'),
 ('P006', 'card', 'Citizens Bank Limited'),
 ('P007', 'card', 'City Bank Limited'),
 ('P008', 'card', 'Community Bank Bangladesh Limited'),
 ('P009', 'card', 'Dhaka Bank Limited'),
 ('P010', 'card', 'Dutch-Bangla Bank Limited'),
 ('P011', 'card', 'Eastern Bank Limited'),
 ('P012', 'card', 'IFIC Bank Limited'),
 ('P013', 'card', 'Jamuna Bank Limited'),
 ('P014', 'card', 'Meghna Bank Limited'),
 ('P015', 'card', 'Mercantile Bank Limited'),
 ('P016', 'card', 'Midland Bank Limited'),
 ('P017', 'card', 'Modhumoti Bank Limited'),
 ('P018', 'card', 'Mutual Trust Bank Limited'),
 ('P019', 'card', 'National Bank Limited'),
 ('P020', 'card', 'National Credit & Commerce Bank Limited'),
 ('P021', 'card', 'NRB Bank Limited'),
 ('P022', 'card', 'NRB Commercial Bank Ltd'),
 ('P023', 'card', 'NRB Global B

In [26]:
trans_df = pd.DataFrame(list(records), columns=['payment_key', 'trans_type', 'bank_name'])
trans_df

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited
